In [1]:
!git clone https://github.com/ArkZ10/APACO.git

Cloning into 'APACO'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 15.37 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import json
import pandas as pd

with open('/content/APACO/multiqc_data_665.json') as f:
    data = json.load(f)


In [3]:
with open('/content/APACO/multiqc_data_379.json') as f:
    data_379 = json.load(f)

general_stats_379 = data_379['report_general_stats_data']

In [4]:
general_stats = data['report_general_stats_data']

flight_samples = {}
hgc_samples = {}
vgc_samples = {}


for entry in general_stats:
  for sample_name, stats in entry.items():
      if "FLT" in sample_name:
          flight_samples[sample_name] = stats
      elif "GC" in sample_name:
          hgc_samples[sample_name] = stats
      elif "VIV" in sample_name:
          vgc_samples[sample_name] = stats

print(f"Flight samples: {len(flight_samples)}")
print(f"HGC samples: {len(hgc_samples)}")
print(f"VGC samples: {len(vgc_samples)}")


Flight samples: 20
HGC samples: 20
VGC samples: 20


In [5]:
import pandas as pd

summary = []

for entry in general_stats:
    for sample, stats in entry.items():
        sample_summary = {
            'Sample': sample,
            'GC Content (%)': stats.get('percent_gc', 'N/A'),
            'Average Sequence Length': stats.get('avg_sequence_length', 'N/A'),
            'Total Sequences': stats.get('total_sequences', 'N/A'),
            'Duplication Rate (%)': stats.get('percent_duplicates', 'N/A'),
            'Failed Sequences (%)': stats.get('percent_fails', 'N/A')
        }
        summary.append(sample_summary)

summary_665_df = pd.DataFrame(summary)
summary_665_df.to_csv('/content/multiqc_summary_665.csv', index=False)

In [6]:
summary = []

for entry in general_stats_379:
    for sample, stats in entry.items():
        sample_summary = {
            'Sample': sample,
            'GC Content (%)': stats.get('percent_gc', 'N/A'),
            'Average Sequence Length': stats.get('avg_sequence_length', 'N/A'),
            'Total Sequences': stats.get('total_sequences', 'N/A'),
            'Duplication Rate (%)': stats.get('percent_duplicates', 'N/A'),
            'Failed Sequences (%)': stats.get('percent_fails', 'N/A')
        }
        summary.append(sample_summary)

summary_379_df = pd.DataFrame(summary)
summary_379_df.to_csv('/content/multiqc_summary_379.csv', index=False)

In [7]:
df = pd.concat([summary_665_df, summary_379_df], ignore_index=True)
df.head()

,Sample,GC Content (%),Average Sequence Length,Total Sequences,Duplication Rate (%),Failed Sequences (%)
0,RR23_R-EDL_FLT_F9_R2_raw,45.0,151.0,98082584.0,75.899569,27.272727
1,RR23_R-EDL_FLT_F2_R2_raw,44.0,151.0,94177785.0,76.702949,27.272727
2,RR23_R-EDL_GC_G6_R1_raw,45.0,151.0,90677361.0,76.868881,18.181818
3,RR23_R-EDL_VIV_V5_R2_raw,45.0,151.0,109488221.0,76.717109,27.272727
4,RR23_R-EDL_GC_G9_R2_raw,45.0,151.0,106644836.0,75.975180,27.272727


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Sample                   342 non-null    object 
 1   GC Content (%)           342 non-null    float64
 2   Average Sequence Length  342 non-null    float64
 3   Total Sequences          342 non-null    float64
 4   Duplication Rate (%)     342 non-null    float64
 5   Failed Sequences (%)     342 non-null    float64
dtypes: float64(5), object(1)
memory usage: 16.2+ KB


In [9]:
mean_duplication_rate = df['Failed Sequences (%)'].mean()

print(f"The mean duplication rate is: {mean_duplication_rate:.2f}%")


The mean duplication rate is: 29.45%


In [10]:
df['quality_label'] = df.apply(lambda row: 0 if (row['Duplication Rate (%)'] > 77.70 or row['Failed Sequences (%)'] > 28) else 1, axis=1)


In [11]:
df['quality_label'].value_counts()

,count
quality_label,
0,197
1,145


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df[['GC Content (%)', 'Total Sequences', 'Duplication Rate (%)', 'Failed Sequences (%)']]
y = df['quality_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 82.61%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
train_accuracy = model.score(X_train, y_train)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

test_accuracy = model.score(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Training Accuracy: 82.42%
Test Accuracy: 82.61%
